In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import copy
import csv
from ESIOS import *
from matplotlib import pyplot as plt
from datetime import datetime
from scipy.stats import pearsonr
from functools import reduce

def variables(fecha_inicio, fecha_fin):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    indicadores = [1, 12, 15, 20, 22, 24, 33, 36, 50, 57, 58, 59, 68, 71, 77, 79, 86, 89, 91, 92, 93, 96, 98, 105, 106, 107, 109, 113, 114, 117, 122, 123, 124, 127, 128, 129, 138, 141, 142, 147, 148, 152, 156, 157, 158, 160, 162, 173, 175, 176, 183, 184, 191, 192, 194, 197, 198, 201, 359, 360, 368, 371, 372, 373, 374, 380, 386, 387, 421, 454, 463, 480, 489, 492, 495, 496, 497, 498, 500, 502, 505, 510, 512, 514, 515, 516, 519, 522, 524, 528, 529, 530, 531, 532, 533, 534, 537, 538, 541, 572, 578, 579, 632, 633, 634, 674, 675, 680, 684, 685, 704, 709, 712, 726, 727, 734, 735, 738, 742, 750, 751, 754, 755, 757, 762, 764, 766, 770, 775, 776, 777, 778, 780, 781, 783, 784, 785, 786, 788, 789, 793, 794, 795, 797, 798, 802, 803, 804, 805, 806, 807, 813, 815, 816, 830, 843, 899, 1014, 1015, 1017, 1019, 1020, 1024, 1025, 1026, 1029, 1031, 1032, 1033, 1034, 1035, 1045, 1048, 1074, 1075, 1079, 1084, 1085, 1086, 1088, 1106, 1113, 1114, 1115, 1116, 1136, 1173, 1177, 1180, 1181, 1189, 1192, 1196, 1197, 1198, 1202, 1204, 1212, 1213, 1216, 1217, 1275, 1276, 1277, 1286, 1334, 1338, 1341, 1342, 1343, 1344, 1349, 1350, 1352, 1354, 1358, 1369, 1373, 1376, 1389, 1397, 1411, 1420, 1421, 1422, 1423, 1425, 1434, 1437, 1438, 1635, 1636, 1637, 1638, 1641, 1642, 1645, 1646, 1647, 1651, 1652, 1656, 1657, 1661, 1663, 1666, 1686, 1687, 1727, 1775, 1776, 1777, 1788, 10008, 10010, 10012, 10013, 10018, 10021, 10023, 10024, 10026, 10027, 10031, 10034, 10047, 10051, 10055, 10056, 10063, 10064, 10067, 10068, 10073, 10086, 10088, 10089, 10095, 10096, 10098, 10105, 10106, 10107, 10108, 10114, 10116, 10133, 10134, 10159, 10169, 10171, 10176, 10177, 10186, 10187, 10188, 10189, 10190, 10207, 10208, 10209, 10211, 10212, 10222, 10223, 10228, 10229, 10230, 10235, 10236, 10241, 10242, 10247, 10249, 10250, 10257, 10258, 10267, 10268, 10270, 10271, 10274, 10275, 10276, 10281, 10286, 10287, 10321, 10324, 10359, 10364, 10380, 10381]
    lista = []
    start = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    end = datetime.strptime(fecha_fin, '%Y-%m-%d')
    dias = end - start
    objetivo = dias.days*24+2  #Puede ser +1 +2 dependiendo del cambio horario
    print('objetivo=' +str(objetivo))
    for i in indicadores:
        datos = esios.get_data(i, start, end)
        if(datos is not None):
            print(str(i) + '=' + str(len(datos)))
            if(len(datos)==objetivo):
                lista.append(i)
    print('Tienes ' + str(len(lista)) + ' variables validas')
    return lista

def nombres_indicadores(indicadores):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    lista_nombres = esios.get_names(indicadores)
    lista_final = dict(zip(lista_nombres,indicadores))
    df = pd.DataFrame.from_dict(lista_final, orient='index')
    df = df.transpose()
    df.to_csv("nombres_indicadores.csv", index=False)

def extraer_datos(fecha_inicio, fecha_fin, nombre, indicadores = []):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    start = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    end = datetime.strptime(fecha_fin, '%Y-%m-%d')
   
    if(not indicadores):

        indicadores = variables(fecha_inicio, fecha_fin)
    
    df_list, names = esios.get_multiple_series(indicadores, start, end)
    
    final = esios.merge_series(df_list, names, pandas_sampling_interval='1H')
    final.to_csv(nombre+'.csv')

def extraer_datos_por_partes(fecha_inicio, fecha_fin, nombre, indicadores = []):
    corte = 5    #Número de variables en cada subgrupo
    lista_indicadores = []
    lista_df = []
    i=0
    e=0
 
    if(not indicadores):
        indicadores = variables(fecha_inicio, fecha_fin)

    lista_indicadores=[indicadores[i:i + corte] for i in range(0, len(indicadores), corte)]
    
    if(len(lista_indicadores[-1])<corte):
        lista_indicadores[-2]=lista_indicadores[-2]+lista_indicadores[-1]
        lista_indicadores.pop()
    print(lista_indicadores)
    
    for i in lista_indicadores:
        nombre_parte = 'Extraer parte '+ str(e)
        extraer_datos(fecha_inicio, fecha_fin, nombre_parte , i)
        e=e+1
        
    for i in range(0,len(lista_indicadores)):
        parte_leer = 'Extraer parte ' + str(i) + '.csv'
        data = pd.read_csv(parte_leer)
        if(i!=0):
            del(data['datetime_utc'])
        lista_df.append(data)  

    df = pd.concat(lista_df,axis=1, join='inner')

    pd.DataFrame.to_csv(df, nombre+'.csv', sep=',', na_rep='.', index=False)
    print('Fin de extraer datos por partes')  
    
def seleccion_variables_pearson(nombre):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    df = pd.read_csv(nombre+".csv")
    nombres = pd.read_csv("nombres_indicadores.csv")
    columnas = df.columns
    objetivo = df['Precio medio horario final suma de componentes']
    lista04 = []
    lista06 = []
    lista075 = []
    for nom in columnas:
        if(not nom=='datetime_utc'):
            columna = df[nom]
            corr, _ = pearsonr(objetivo, columna)
            print('%.3f' % corr+' : '+nom)
            if(abs(corr) >= 0.75):
                ind = nombres.get(nom)
                lista075.append(int(ind[0]))
                lista06.append(int(ind[0]))
                lista04.append(int(ind[0]))
            elif(abs(corr) >= 0.6):
                ind = nombres.get(nom)
                lista06.append(int(ind[0]))
                lista04.append(int(ind[0]))
            elif(abs(corr) >= 0.4):
                ind = nombres.get(nom)
                lista04.append(int(ind[0]))
    return lista075, lista06, lista04

def MAE (test_y, predicciones):
    errores=[]
    for i in range(0, len(test_y)):
        errores.append(abs(test_y[i]-predicciones[i]))
    return np.mean(errores)

def MAPE (test_y, predicciones):
    errores=[]
    for i in range(0, len(test_y)):
        errores.append(abs(test_y[i]-predicciones[i])/test_y[i]*100)
    return np.mean(errores)

def WAPE (test_y, predicciones): 
    actual=sum(test_y)
    pred=sum(predicciones)   
    return float(abs(actual-pred)/actual*100)

def resultado(nombre, algoritmo, num):
    MAE=[]
    MAPE=[]
    WAPE=[]
    for i in range(0,num):
        print("Iteración "+str(i+1))
        resultadoMAE,resultadoMAPE,resultadoWAPE=algoritmo(nombre)
        MAE.append(resultadoMAE)
        MAPE.append(resultadoMAPE)
        WAPE.append(resultadoWAPE)
    resultado=[np.mean(MAE),np.mean(MAPE),np.mean(WAPE)]
    print("================================")
    print("Media final: \nError MAE: "+str(resultado[0])+"\nError MAPE: "+str(resultado[1])+"\nError WAPE: "
          +str(resultado[2]))
    return resultado 

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from tcn import TCN, tcn_full_summary
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow
from matplotlib import pyplot


def random_forest(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
        
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    modelo = RandomForestRegressor(n_estimators=30,criterion='mse',
                                    max_depth=None, max_features='auto',
                                   oob_score = False,n_jobs = -1, random_state=1234)

    modelo.fit(train_X, train_y)

    predicciones = modelo.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE

def multilayer_perceptron(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    modelo = MLPRegressor(hidden_layer_sizes=(32,32,32,32,32),activation="relu" ,random_state=1, max_iter=2000)
    modelo.fit(train_X, train_y)

    predicciones = modelo.predict(test_X)
    
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE

def tcn(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]

    time_steps, input_dim = 20, 1
    tcn_layer = TCN(60,input_shape=(time_steps, input_dim))

    modelo = Sequential([tcn_layer, Dense(1)])

    modelo.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_absolute_error')
    history = modelo.fit(train_X, train_y, epochs=150, validation_data=(test_X, test_y))

    predicciones = modelo.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    print(modelo.summary())

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()
    
    return errorMAE,errorMAPE,errorWAPE

In [6]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
import keras
from hyperband import HyperbandSearchCV
from sklearn.model_selection import GridSearchCV

def random_forest_rs(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
        
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    
    
    estimator=RandomForestRegressor()


    param_grid = {'n_estimators': [25 ,50, 75, 100, 150],
          'criterion': ['squared_error','absolute_error','poisson','mse'],
          'max_depth': [None, 5, 10, 20],
          'min_samples_split': [1, 5, 10],
          'min_samples_leaf': [1, 3, 6],
          'max_features': ['auto','sqrt','log2', None, 5, 10]
          }

    gsc = GridSearchCV(estimator,param_grid,cv=2, verbose=3, n_jobs=4)

    grid_result = gsc.fit(train_X, train_y)


    best_params = grid_result.best_params_

    best_rf = RandomForestRegressor(n_estimators=best_params["n_estimators"],criterion=best_params["criterion"],
                                    max_depth=best_params["max_depth"], min_samples_split=best_params["min_samples_split"],
                                    min_samples_leaf=best_params["min_samples_leaf"], max_features=best_params["max_features"],
                                    random_state = 1234)
    
    best_rf.fit(train_X, train_y)
   
    print(best_params["n_estimators"])
    print(best_params["criterion"])
    print(best_params["max_depth"])
    print(best_params["min_samples_split"])
    print(best_params["min_samples_leaf"])
    print(best_params["max_features"])
    
    predicciones = best_rf.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))

    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))

    return errorMAE,errorMAPE,errorWAPE


def random_forest_hp(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
        
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    
    
    estimator=RandomForestRegressor()


    param_grid = {'n_estimators': [25,50, 75, 100, 150],
          "criterion": ['mse','mae'],
          'max_depth': [None, 5, 10, 20],
          'min_samples_split': [1.0, 5, 10],
          'min_samples_leaf': [1, 3, 6],
          'max_features': ['auto','sqrt','log2', None, 5, 10]
          }
    print(param_grid)
    gsc = HyperbandSearchCV(cv=4,estimator=estimator, param_distributions=param_grid,resource_param='min_samples_split', scoring='neg_mean_squared_error',verbose=3,n_jobs=4,max_iter=500)

    grid_result = gsc.fit(train_X, train_y)


    best_params = grid_result.best_params_

    best_rf = RandomForestRegressor(n_estimators=best_params["n_estimators"],criterion=best_params["criterion"],
                                    max_depth=best_params["max_depth"], min_samples_split=best_params["min_samples_split"],
                                    min_samples_leaf=best_params["min_samples_leaf"], max_features=best_params["max_features"], random_state = 1234)
    
    best_rf.fit(train_X, train_y)
   
    print(best_params["n_estimators"])
    print(best_params["criterion"])
    print(best_params["max_depth"])
    print(best_params["min_samples_split"])
    print(best_params["min_samples_leaf"])
    print(best_params["max_features"])
    
    predicciones = best_rf.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))

    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))

    return errorMAE,errorMAPE,errorWAPE

In [ ]:
import time
print(time.strftime("%X")+" Inicio random forest rs 1 075")
random_forest_rs("Primer periodo 075")
print(time.strftime("%X")+" final random forest rs 1 075 e inicio random forest hp 1 075")
random_forest_hp("Primer periodo 075")
print(time.strftime("%X")+" final random forest hp 1 075")

In [8]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
import numpy as np
from hyperband import HyperbandSearchCV
from sklearn.model_selection import RandomizedSearchCV

def multilayer_perceptron_rs(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc") 
    values = df.values
    names=df.columns
    indice=names.get_loc("Precio medio horario final suma de componentes")
    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y=train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X= train[:, :]
    test_y=test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]

    estimator=MLPRegressor()


    param_grid = {'hidden_layer_sizes': [(32,32,32,32,32), (64,64,64,64,64), (100,100,100), (75, 75, 75, 75), (50,100,50), (100,)],
          'activation': ['relu', 'tanh', 'logistic', 'identity'],
          'alpha': [0.0001, 0.001, 0.01],
          'solver':['lbfgs'],
          'learning_rate': ['constant','adaptive', 'invscaling'],
          'shuffle': [True, False],
          'max_iter':[2000]}

    gsc = RandomizedSearchCV(estimator, param_grid, cv=4, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

    grid_result = gsc.fit(train_X, train_y)

    best_params = grid_result.best_params_

    best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], 
                        activation =best_params["activation"],
                        alpha = best_params["alpha"],
                        learning_rate=best_params["learning_rate"],
                        solver='lbfgs',
                        shuffle=best_params["shuffle"],
                        batch_size='auto',
                        max_iter=10000)
    
    best_mlp.fit(train_X, train_y)
    
    print("hidden_layer_sizes: "+str(best_params.get('hidden_layer_sizes')))
    print("activation: "+str(best_params.get('activation')))
    print("alpha: "+str(best_params.get('alpha')))
    print("learning_rate: "+str(best_params.get('learning_rate')))
    print("shuffle: "+str(best_params.get('shuffle')))
    
    predicciones = best_mlp.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE


def multilayer_perceptron_hp(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc") 
    values = df.values
    names=df.columns
    indice=names.get_loc("Precio medio horario final suma de componentes")
    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y=train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X= train[:, :]
    test_y=test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]

    estimator=MLPRegressor()


    param_grid = {'hidden_layer_sizes': [(32,32,32,32,32), (64,64,64,64,64), (100,100,100), (75, 75, 75, 75), (50,100,50), (100,)],
          'activation': ['relu', 'tanh', 'logistic', 'identity'],
          'alpha': [0.0001, 0.001, 0.01],
          'solver':['lbfgs'],
          'learning_rate': ['constant','adaptive', 'invscaling'],
          'shuffle': [True, False],
          'max_iter':[2000]}

    gsc = HyperbandSearchCV(cv=3,estimator=estimator, param_distributions=param_grid,resource_param='n_iter_no_change', scoring='neg_mean_squared_error',verbose=3,n_jobs=4,max_iter=500)

    grid_result = gsc.fit(train_X, train_y)

    best_params = grid_result.best_params_

    best_mlp = MLPRegressor(hidden_layer_sizes = best_params["hidden_layer_sizes"], 
                        activation =best_params["activation"],
                        alpha = best_params["alpha"],
                        learning_rate=best_params["learning_rate"],
                        solver='lbfgs',
                        shuffle=best_params["shuffle"],
                        batch_size='auto',
                        max_iter=10000)
    
    best_mlp.fit(train_X, train_y)
    
    print("hidden_layer_sizes: "+str(best_params.get('hidden_layer_sizes')))
    print("activation: "+str(best_params.get('activation')))
    print("alpha: "+str(best_params.get('alpha')))
    print("learning_rate: "+str(best_params.get('learning_rate')))
    print("shuffle: "+str(best_params.get('shuffle')))
    
    predicciones = best_mlp.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE




In [ ]:
import time
print(time.strftime("%X")+" Inicio multilayer perceptron rs 1 075")
multilayer_perceptron_rs("Primer periodo 075")
print(time.strftime("%X")+" Final multilayer perceptron rs 1 075 e inicio multilayer perceptron hp 1 075")
multilayer_perceptron_hp("Primer periodo 075")
print(time.strftime("%X")+" Final multilayer perceptron hp 1 075")

In [9]:
import keras_tuner as kt
import pandas as pd
import numpy as np
from tcn import TCN, tcn_full_summary
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from keras_tuner.tuners import Hyperband, BayesianOptimization, RandomSearch


def tcn_rs(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    

    tuner = RandomSearch(model_builder_rs,
                     objective='loss',
                     directory='prueba',
                     project_name='TCN-randomsearch106',
                     max_trials=50)
    stop_early = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    
    tuner.search(train_X, train_y, epochs=150, validation_data=(test_X, test_y),callbacks=[stop_early])
    best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]
    print("nb_filters: "+str(best_hps.get('nb_filters')))
    print("learning_rate: "+str(best_hps.get('learning_rate')))
    print("kernel_size: "+str(best_hps.get('kernel_size')))
    print("nb_stacks: "+str(best_hps.get('nb_stacks')))
    print("dropout_rate: "+str(best_hps.get('dropout_rate')))
    print("use_layer_norm: "+str(best_hps.get('use_layer_norm')))

    
def model_builder_rs(hp):
    
    
    time_steps, input_dim = 20, 1
    hp_nb_filters = hp.Int('nb_filters', min_value=20, max_value=140, step=10)
    hp_kernel_size = hp.Int('kernel_size',min_value=1,max_value=9,step=2)
    hp_nb_stacks = hp.Int('nb_stacks',min_value=1,max_value=5,step=2)
    hp_dropout_rate=hp.Float('dropout_rate',min_value=0.0,max_value=0.25,step=0.05)
    hp_use_layer_norm=hp.Choice('use_layer_norm', values = [False,True])
    
    tcn_layer = TCN(nb_filters=hp_nb_filters, kernel_size=hp_kernel_size, nb_stacks=hp_nb_stacks, 
                    dropout_rate=hp_dropout_rate, use_layer_norm=hp_use_layer_norm, input_shape = (time_steps, input_dim))

    modelo = Sequential([tcn_layer, Dense(1)])
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    modelo.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mean_absolute_error')
    return modelo



def tcn_bo(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    

    tuner = BayesianOptimization(model_builder_bo,
                     objective='loss',
                     directory='prueba',
                     project_name='TCN-bayesian106',
                     max_trials=75)
    stop_early = keras.callbacks.EarlyStopping(monitor='loss', patience=6)
    
    tuner.search(train_X, train_y, epochs=150, validation_data=(test_X, test_y),callbacks=[stop_early])
    best_hps=tuner.get_best_hyperparameters(num_trials=10)[0]
    print("nb_filters: "+str(best_hps.get('nb_filters')))
    print("learning_rate: "+str(best_hps.get('learning_rate')))
    print("kernel_size: "+str(best_hps.get('kernel_size')))
    print("nb_stacks: "+str(best_hps.get('nb_stacks')))
    print("dropout_rate: "+str(best_hps.get('dropout_rate')))
    print("use_layer_norm: "+str(best_hps.get('use_layer_norm')))

    
def model_builder_bo(hp):
    
    
    time_steps, input_dim = 20, 1
    hp_nb_filters = hp.Int('nb_filters', min_value=20, max_value=140, step=10)
    hp_kernel_size = hp.Int('kernel_size',min_value=1,max_value=9,step=2)
    hp_nb_stacks = hp.Int('nb_stacks',min_value=1,max_value=5,step=2)
    hp_dropout_rate=hp.Float('dropout_rate',min_value=0.0,max_value=0.25,step=0.05)
    hp_use_layer_norm=hp.Choice('use_layer_norm', values = [False,True])
    
    tcn_layer = TCN(nb_filters=hp_nb_filters, kernel_size=hp_kernel_size, nb_stacks=hp_nb_stacks, 
                    dropout_rate=hp_dropout_rate, use_layer_norm=hp_use_layer_norm, input_shape = (time_steps, input_dim))

    modelo = Sequential([tcn_layer, Dense(1)])
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    modelo.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='mean_absolute_error')
    return modelo

In [ ]:
tcn_rs("Primer periodo 075")

In [ ]:
tcn_bo("Primer periodo 075")